#### **Domain**: Data Science
#### **Project Title**: Student Movement Analysis
#### **Team Members**:
* CS21B1039 MUPPA PRABHAS REDDY 
* CS21B1038 DESETTI RAJESH
#### **Institution**: Indian Institute of Information Technology, Design & Manufacturing, Kancheepuram

#### **Data Collection**
* **Source**: Student Movement Register
* **Raw Data File**: raw_student_movement.csv
* **Cleaned Data File**: cleaned_student_movement.csv

#### **Data Analysis Steps
1.Data collection
2.Data preparation
    2.1.Data Discretization
    	1.Data binning
    	2.clustering
    	3.Classification
    2.2.Data cleaning
    	1.remove duplicates
    	2.remove irrelevent data
    	3.handle incomplete data
    	4.remove outliers
    	5.fix structural errors	
    2.3.Data integration
    2.4.Data Transformation
    	1.smoothing
    	2.aggregation
    	3.genralization
    	4.normalisation
    2.5.Data reduction

#### **Data Loading into DataFrame**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('raw_student_movement.csv')
data_types = df.dtypes
print("Data Types of Attributes\n........................")
print(data_types)
print("\nData: Student Movement Details\nSource: Student Movement Register\n")
df

#### **Type Conversion of Time to Time in Seconds**

In [ ]:
# Function to convert time to seconds
def time_to_seconds(time_str):
    # Split the time string by ':'
    time_parts = time_str.split(':')
    # Convert hours and minutes to seconds and add them up
    total_seconds = int(time_parts[0]) * 3600 + int(time_parts[1]) * 60
    return total_seconds

# Apply the function to the 'OutTime' and 'InTime' column and create new 'OutTimeSeconds' and 'InTimeSeconds'columns
df['OutTimeSeconds'] = df['OutTime'].apply(time_to_seconds)
df['InTimeSeconds'] = df['InTime'].apply(time_to_seconds)

df

#### **Data Preparation**

#### **1. Data Discretization** - Binning

In [ ]:
# create a dataframe to store the binned column- TimeOfDay
binned_df = pd.DataFrame()
binned_df = df[['RollNumber', 'OutTimeSeconds']].copy()

# declare the number of bins and their width
# 00:00 to 11:59 - Morning
# 12:00 to 17:59 - Afternoon
# 18:00 to 23:59 - Evening

bin_edges = [0, 43200, 64800, 86400]
bin_labels = ['Morning', 'Afternoon', 'Evening']

binned_df['TimeOfDay'] = pd.cut(df['OutTimeSeconds'], bins=bin_edges, labels=bin_labels)
print(f"Total number of bins: {len(bin_labels)}")
print("Bins: ", end=" ")
for label in bin_labels:
    print(label, end=" ")
binned_df

#### **2. Data Cleaning**

#### **2.1. Removing Duplicates** - drop_duplicates()

In [ ]:
# Count the number of duplicated rows
num_of_duplicates = df.duplicated().sum()
print(f"Number of duplicated rows: {num_of_duplicates}\n")

# drop the duplicated rows from the dataframe
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df

#### **2.2. Removing Irrelevant Data** - drop()

In [ ]:
# drop irrelevant columns from the dataframe
irrelevant_columns = ['RoomNumber', 'PhoneNumber']
df.drop(columns=irrelevant_columns, inplace=True)
print("Irrelevant columns dropped: ", end=" ")
for column in irrelevant_columns:
    print(column, end=" ")
print("\n")
df

#### **2.3. Handling Missing Data** - fillna() and dropna()

In [ ]:
# Check for missing values
print("Number of missing values in each column >> BEFORE DROP")
print(df.isnull().sum())

# check for null values in the column VisitingPlace
null_count_visitingPlace = df['VisitingPlace'].isnull().sum()
print(f"Number of null values in the column VisitingPlace: {null_count_visitingPlace}")

# fill the null values in VisitingPlace column with mode of that column
print(f"\nMode of VisitingPlace column: {df['VisitingPlace'].mode()[0]}")
df['VisitingPlace'].fillna(df['VisitingPlace'].mode()[0], inplace=True)

# Drop rows with any missing values
df.dropna(inplace=True)
print("Number of missing values in each column >> AFTER DROP")
print(df.isnull().sum())
# Interpolate missing values using linear method
# df['OutTimeSeconds'].interpolate(method='linear', inplace=True)
# df['InTimeSeconds'].fillna(df['InTimeSeconds'].mean(), inplace=True)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
print("Statistical description of numerical columns\n")
df.describe()

#### **2.4. Detecting Outliers** - Box Plot

In [ ]:
def time_format(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    return '{:02d}:{:02d}'.format(int(hours), int(minutes))

# function to get the minimum and maximum value of boxplot
def get_boxplot_min_max(dataframe, column_name):
    # Get the descriptive statistics of the column
    column_stats = dataframe[column_name].describe()

    # Calculate the interquartile range (IQR)
    Q1 = column_stats['25%']
    Q3 = column_stats['75%']
    IQR = Q3 - Q1
    print(f"25 Percentile: {time_format(Q1)} ({int(Q1)})\n75 Percentile: {time_format(Q3)} ({int(Q3)})")
    print(f"Inter Quartile Range: {time_format(IQR)} ({int(IQR)})")

    # Calculate the minimum and maximum values for the boxplot
    boxplot_min = Q1 - 1.5 * IQR
    boxplot_max = Q3 + 1.5 * IQR

    return boxplot_min, boxplot_max

min_value, max_value = get_boxplot_min_max(df, 'OutTimeSeconds')
if min_value < 0:
    print(f"Minimum value for boxplot: 00:00 (0)")
else:
    print(f"Minimum value for boxplot: {time_format(min_value)}({int(min_value)})")
if max_value >= 86400:
    print(f"Maximum value for boxplot: 23:59 (86400)")
else:
    print(f"Maximum value for boxplot: {time_format(max_value)}({int(max_value)})")
df = df[(df['OutTimeSeconds'] >= min_value) & (df['OutTimeSeconds'] <= max_value)]
df.reset_index(drop=True, inplace=True)
df

#### **2.5. Fixing Structural Errors**

In [ ]:

df.loc[df['Hostel'] == 'Jasmine', 'Hostel'] = 'Jasmine Annex'
df

# data = {'RollNumber': ['CS21B1039'], 'Hostel': ['Jasmine'], 'VisitingPlace':['Kandigai'], 'OutTime': ['17:20'],
            # 'InTime': ['18:30'],'OutTimeSeconds': [48000], 'InTimeSeconds': [64000], 'TimePeriod': ['Evening'], 'TimePeriodNumeric': [2] , 'Cluster': [2]}
# temp_df = pd.DataFrame(data)
# df1 = df.copy()
# merged_df = pd.concat([df1, temp_df], ignore_index=True)
# merged_df['Hostel'] = merged_df['Hostel'].replace('Jasmine', 'Jasmine Annex')
# merged_df

#### **3. Data Integration**

##### Not Applicable

#### **4. Data Transformation**

#### **4.1 Data Smoothing**

In [ ]:
window_size = 5

smoothed_df = pd.DataFrame()
smoothed_df['OutTimeSeconds'] = df['OutTimeSeconds'].copy()
smoothed_df['SmoothedOutTimeSeconds'] = df['OutTimeSeconds'].rolling(window=window_size).mean()
smoothed_df.dropna(inplace=True)

plt.hist(smoothed_df['OutTimeSeconds'], bins=150, color='skyblue')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of OutTime (in seconds)')
plt.grid(True)
plt.show()

plt.hist(smoothed_df['SmoothedOutTimeSeconds'], bins=150, color='skyblue')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram of smoothed OutTime (in seconds)')
plt.grid(True)
plt.show()

#### **4.2. Data Aggregation** - Not Applicable

#### **4.3. Data Generalization**

In [ ]:
# Define time ranges for different parts of the day
time_ranges = {
    'Morning': (0, 43200),    # 00:00 to 12:00 (Morning)
    'Afternoon': (43200, 64800),  # 12:00 to 18:00 (Afternoon)
    'Evening': (64800, 86400)  # 18:00 to 24:00 (Evening)
}

# Function to map time to the corresponding part of the day
def get_time_of_day(seconds):
    for time_of_day, (start, end) in time_ranges.items():
        if start <= seconds < end:
            return time_of_day
    return None

# Apply the function to create new 'OutTimeOfDay' and 'InTimeOfDay' columns
# df['OutTimeOfDay'] = df['OutTimeSeconds'].apply(get_time_of_day)
# df['InTimeOfDay'] = df['InTimeSeconds'].apply(get_time_of_day)

# Use .loc to set the values properly without warnings
df = df.copy()
df.loc[:, 'OutTimeOfDay'] = df['OutTimeSeconds'].apply(get_time_of_day)
df = df.copy()
df.loc[:, 'InTimeOfDay'] = df['InTimeSeconds'].apply(get_time_of_day)
df

# timeofday_df = df[['RollNumber', 'OutTimeOfDay', 'InTimeOfDay']].copy()
# timeofday_df

In [ ]:
# Function to get the batch of student from the roll number
def get_batch(roll_number):
    symbols = list(roll_number)
    batch = ''
    for symbol in symbols:
        if symbol >= '0' and symbol <= '9':
            batch += symbol
            if(len(batch) >= 2):
                break
    return int('20' + batch)
    
# Apply the function to create the new 'Batch' column
df = df.copy()
df.loc[:, 'Batch'] = df['RollNumber'].apply(get_batch)
df

#### **4.4. Data Normalization** - Min Max Normalization

In [ ]:
# columns_to_normalize = ['OutTimeSeconds', 'InTimeSeconds']
# def min_max_normalize(column):
#     max_value = column.max()
#     min_value = column.min()
#     normalized_column = (column - min_value) / (max_value - min_value)
#     return normalized_column

# for column in columns_to_normalize:
#     df[column+'Normailzed'] = min_max_normalize(df[column])

# df

#### **Save the cleaned dataset to a new file**

In [ ]:
# df.to_csv('cleaned_student_movement.csv', index=False)
print("Cleaned dataset saved successfully!")

#### **Exploratory Data Analysis**

* #### **Outing Duration Analysis**

In [ ]:
df['Duration'] = df['InTimeSeconds'] - df['OutTimeSeconds']
duration_stats = df['Duration'].describe()
avg_duration = duration_stats['mean']
print("Average duration of outing: ",time_format(avg_duration), int(avg_duration))
min_duration = duration_stats['min']
print("Minimum duration of outing: ",time_format(min_duration), int(min_duration))
max_duration = duration_stats['max']
print("Maximum duration of outing: ",time_format(max_duration), int(max_duration))

plt.hist(df['Duration'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('Duration of Outings')
plt.ylabel('Frequency')
plt.title('Histogram of Outing Durations')
plt.show()

import seaborn as sns

# Distribution plot (histogram with a KDE)
sns.histplot(df['Duration'], kde=True, color='orange')
plt.xlabel('Duration of Outings')
plt.ylabel('Density')
plt.title('Distribution of Outing Durations')
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(df['OutTimeSeconds'], df['Duration'], marker='o', linestyle='-')
plt.xlabel('Timestamp')
plt.ylabel('Duration of Outings')
plt.title('Outing Durations Over Time')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

#### **Heatmap** - Correlation between OutTime, InTime and Duration

In [ ]:
# Create a heatmap to visualize the correlation
correlation_matrix = df[['OutTimeSeconds', 'InTimeSeconds', 'Duration']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap between OutTime, InTime and Duration')
plt.show()

* #### **Scatter Plot** - Holistic view of data distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 1 row, 2 columns for subplots

# Scatter plot for OutTime vs. InTime by OutTimeOfDay
out_morning = df.loc[df['OutTimeOfDay'] == 'Morning', ['OutTimeSeconds', 'InTimeSeconds']]
out_afternoon = df.loc[df['OutTimeOfDay'] == 'Afternoon', ['OutTimeSeconds', 'InTimeSeconds']]
out_evening = df.loc[df['OutTimeOfDay'] == 'Evening', ['OutTimeSeconds', 'InTimeSeconds']]

axes[0].scatter(out_morning['OutTimeSeconds'], out_morning['InTimeSeconds'], label='morning', marker='o', color='tab:orange')
axes[0].scatter(out_afternoon['OutTimeSeconds'], out_afternoon['InTimeSeconds'], label='afternoon', marker='x', color='tab:blue')
axes[0].scatter(out_evening['OutTimeSeconds'], out_evening['InTimeSeconds'], label='evening', marker='^', color='tab:green')
axes[0].set_xlabel('OutTime')
axes[0].set_ylabel('InTime')
axes[0].set_title('OutTime vs. InTime by OutTime of Day')
axes[0].legend()

# Scatter plot for OutTime vs. InTime by InTimeOfDay
in_morning = df.loc[df['InTimeOfDay'] == 'Morning', ['OutTimeSeconds', 'InTimeSeconds']]
in_afternoon = df.loc[df['InTimeOfDay'] == 'Afternoon', ['OutTimeSeconds', 'InTimeSeconds']]
in_evening = df.loc[df['InTimeOfDay'] == 'Evening', ['OutTimeSeconds', 'InTimeSeconds']]

axes[1].scatter(in_morning['OutTimeSeconds'], in_morning['InTimeSeconds'], label='morning', marker='o', color='tab:orange')
axes[1].scatter(in_afternoon['OutTimeSeconds'], in_afternoon['InTimeSeconds'], label='afternoon', marker='x', color='tab:blue')
axes[1].scatter(in_evening['OutTimeSeconds'], in_evening['InTimeSeconds'], label='evening', marker='^', color='tab:green')
axes[1].set_xlabel('OutTime')
axes[1].set_ylabel('InTime')
axes[1].set_title('OutTime vs. InTime by InTime of Day')
axes[1].legend()

plt.tight_layout()  # To prevent overlapping of plots
plt.show()

* #### Count of OutTime and InTime by Time of Day

In [ ]:
# Grouping by 'InTimeOfDay' and 'OutTimeOfDay' and counting occurrences
in_time_counts = df['InTimeOfDay'].value_counts()
out_time_counts = df['OutTimeOfDay'].value_counts()

# Creating a figure and two subplots for side-by-side bar plots
fig, ax = plt.subplots()

# Bar width for each set of bars
bar_width = 0.35

# Generating positions for bars
x = range(len(out_time_counts))

# Plotting the bars
ax.bar(x, out_time_counts, width=bar_width, label='Out')
ax.bar([pos + bar_width for pos in x], in_time_counts, width=bar_width, label='In')

# Setting labels and title
ax.set_xlabel('Time of Day')
ax.set_ylabel('Count')
ax.set_title('Count of OutTime and InTime by Time of Day')
ax.set_xticks([pos + bar_width / 2 for pos in x])
ax.set_xticklabels(in_time_counts.index)
ax.legend()

plt.show()

* #### Hostel Distribution

In [ ]:
# Count the occurrences of each unique value in the 'Hostel' column
hostel_counts = df['Hostel'].value_counts()
print(hostel_counts)
# create bar chart for hostel
hostel_counts.plot(kind='bar', color='mediumpurple')
plt.title('Hostel Distribution')
plt.xlabel('Hostel')
plt.ylabel('Count')

plt.show()

* #### Visiting Place Distribution

In [ ]:
# Count the occurrences of each unique value in the 'VistingPlace' column
place_counts = df['VisitingPlace'].value_counts()
print(place_counts)

# create bar chart for visiting place
place_counts.plot(kind='bar', color='mediumseagreen')
plt.title('Visting Place Distribution')
plt.xlabel('Visting Place')
plt.ylabel('Count')
plt.show()

* #### Percentage of Outings by Batch

In [ ]:
# Counting the number of records for each batch
batch_counts = df['Batch'].value_counts()
print(batch_counts)

# Plotting a pie chart showing the percentage of outings for each batch
plt.figure(figsize=(6, 6))
plt.pie(batch_counts, labels=batch_counts.index, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Percentage of Outings by Batch')
plt.show()